In [6]:
## 
import torch
from torch import nn
import pandas as pd
import os
import numpy as np
import COSMO_TL as ctl
from dask.distributed import Client, LocalCluster, progress
import dask
from scipy.interpolate import RegularGridInterpolator
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import interpn
from scipy.interpolate import LinearNDInterpolator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import  mean_squared_error, r2_score, mean_absolute_error
from scipy.optimize import curve_fit, minimize, differential_evolution
# get particle swarm optimizer
from pyswarm import pso
import time
import os
import pickle
import datetime
import kmodels as kmk
import shutil
import dask.dataframe as dd


def get_X_solute(df):
    X = df[['volume_solute', 'area_solute', 'NC_K', 'SIGMA_K','TAU', 'default_error']]
    sig_cols = [col for col in df.columns if 'sigma_solute' in col]
    sigs = df[sig_cols].to_numpy()
    X = X.to_numpy().reshape(len(df), -1)
    X = np.column_stack((X, sigs))
    return X

def get_X_solvent(df):
    X = df[['volume_solvent', 'area_solvent','NC_K','SIGMA_K','TAU', 'default_error']]
    sig_cols = [col for col in df.columns if 'sigma_solvent' in col]
    sigs = df[sig_cols].to_numpy()
    X = X.to_numpy().reshape(len(df), -1)
    X = np.column_stack((X, sigs))
    return X

def get_X(df):
    X_solute = get_X_solute(df)
    X_solvent = get_X_solvent(df)
    # solvent prop cols = eps,n,alpha,beta,gamma,phi**2,psi**2,beta**2
    solvent_props_names = ['eps', 'n', 'alpha', 'beta', 'gamma', 'phi**2', 'psi**2', 'beta**2']
    solvent_props = df[solvent_props_names].to_numpy()
    X = np.column_stack((X_solute, X_solvent, solvent_props))
    return X
# given a dataframe return a dataframe with the mean value for all columns with error in the name
# grouped by SoluteName
def get_mean_df(df):
    df2 = df.copy()
    original_columns = list(df2.columns)
    print(original_columns)
    cols = [col for col in df2.columns if 'error' in col]
    original_columns = [col for col in original_columns if col not in cols]
    df3 = df2.groupby(['SoluteName', 'NC_K','SIGMA_K','TAU'])[cols].mean()
    df3 = df3.reset_index()
    # return a dataframe with the unique values in SoluteName and the mean values for all columns with error in the name
    # get all the other colums  from the original dataframe
    df4 = df2[original_columns]
    df4 = df4.reset_index(drop=True)
    df5 = pd.merge(df4, df3, on=['SoluteName', 'NC_K','SIGMA_K','TAU'])
    df5 = df5.drop_duplicates()
    return df5

csv_path = '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/data/vaspsol_data_3_2_2023_balanced'

#df = pd.read_csv(csv_path)
df = dd.read_parquet(csv_path)
df = df.compute()
print(df)
df['error'] = df['error'].abs()
df = df[df['error'] < 10]
df = df[df['Solvent'] == 'water']
df = df[df['Charge'] == 0]
NC_K_default = 0.0025
SIGMA_K_default = 0.6
TAU_default = 0.000525
# default_df = df[(df['NC_K'] == NC_K_default) & (df['SIGMA_K'] == SIGMA_K_default) & (df['TAU'] == TAU_default)]
# print(default_df)



# df_to_append = default_df[['SoluteName','error']]
# # rename error to default_error
# df_to_append = df_to_append.rename(columns={'error': 'default_error'})
# df_to_append


# # match up the default error back to the original dataframe
# df = pd.merge(df, df_to_append, on=['SoluteName'])
# # this expanded the number of rows in the dataframe. This is not what we want
#df = df.drop_duplicates('Unnamed: 0')

groups = df[df['Solvent']=='water'].groupby(['NC_K', 'SIGMA_K', 'TAU'])
# print(df)

#df_test = pd.read_csv(csv_path)
df_test = dd.read_parquet(csv_path)
df_test = df_test.compute()
# we want the NC_K, SIGMA_K and TAU combinations that are not in 
# the training set
df_test = df_test[~df_test[['NC_K', 'SIGMA_K', 'TAU']].isin(df[['NC_K', 'SIGMA_K', 'TAU']]).all(axis=1)]
default_df = df_test[(df_test['NC_K'] == NC_K_default) & (df_test['SIGMA_K'] == SIGMA_K_default) & (df_test['TAU'] == TAU_default)]
print(default_df)

# get the number of unique groups
# using the groups split of the dataframe so that unique combos of NC_K, SIGMA_K, and TAU are in each group
split = 0.99
## get the unique groups
#groups = df.groupby(['NC_K', 'SIGMA_K', 'TAU'])
# get the indicies of the groups
indicies = [np.array(i) for i in groups.indices.values()]
# get the number of groups
num_groups = len(indicies)
print('Number of groups: ', num_groups)

# get the number of groups to use for training
num_train_groups = int(num_groups*split)
print('Number of groups to use for training: ', num_train_groups)
print('Number of groups to use for testing: ', num_groups - num_train_groups)
# get the indicies of the groups to use for training
print(len(indicies))

idx_temp = np.arange(len(indicies))
train_indicies = [indicies[i] for i in np.random.choice(idx_temp, size=num_train_groups, replace=False)]
train_indicies = np.concatenate(train_indicies)
# get the indicies of the groups to use for testing
test_indicies = np.array([i for i in np.concatenate(indicies) if i not in train_indicies])
train_df = df.iloc[train_indicies]
test_df = df.iloc[test_indicies]

X_train = get_X_solute(train_df)
X_test = get_X_solute(test_df)
y_train = train_df['error'].to_numpy()
y_test = test_df['error'].to_numpy()
X_test = get_X_solute(test_df)

# print out the shape of the training data and the training labels. Nice retro looking print statment
print(f'X_train shape: {X_train.shape}, y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}, y_test shape: {y_test.shape}')
n_observations_train = X_train.shape[0]
n_features_train = X_train.shape[1]
n_observations_test = X_test.shape[0]
n_features_test = X_test.shape[1]

print('TRAINING SET DETAILS')
print(f'Number of observations: {n_observations_train}')
print(f'Number of features: {n_features_train}')

print('TESTING SET DETAILS')
print(f'Number of observations: {n_observations_test}')
print(f'Number of features: {n_features_test}')



scaler  = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print(X_train.shape)
# send the training data to gpu
X_train = torch.from_numpy(X_train).float().reshape(-1, n_features_train)
y_train = torch.from_numpy(y_train).float().reshape(-1, 1)
X_test = torch.from_numpy(X_test).float().reshape(-1, n_features_train)


       Unnamed: 0 Solvent FileHandle   index_0   index_1   index_2  \
0            1850   water    0044met  0.000400  0.100000  0.000525   
1            2818   water    0076but  0.000400  0.100000  0.000525   
2            2818   water    0076but  0.000400  0.100000  0.000525   
3            4824   water    0062dio  0.000400  0.100000  0.000525   
4            5208   water    0086eth  0.000400  0.100000  0.000525   
...           ...     ...        ...       ...       ...       ...   
80195        5430   water    0217wat  0.005504  0.722813  0.003820   
80196        2678   water    0076but  0.005504  0.722813  0.003820   
80197        4685   water    0062dio  0.005504  0.722813  0.003820   
80198        2678   water    0076but  0.005504  0.722813  0.003820   
80199        2678   water    0076but  0.005504  0.722813  0.003820   

       Solvation_Energy  Total_Energy   No.   SoluteName  ...  \
0             -0.080986    -30.286090  2133     methanol  ...   
1             -0.005511    -7

In [11]:
# lets make a new auto encoder that uses n_dim to make a n_layers with n_dim in the middle
# lets make a function that given a desired depth will report a appropriate topology 
# example 3 layers 11 features would be 11, 7, 3

def get_topology(n_features, depth, out_features=3):
    topology = []
    for i in range(depth):
        if i == 0:
            topology.append(n_features)
        else:
            topology.append(int(topology[i-1] / 2))
            if topology[i] < out_features:
                topology[i] = out_features
    topology.append(out_features)
    return topology

# test
get_topology(11, 3,)
4284/204
class autoNN(nn.Module):
    def __init__(self, n_inputs=11, n_dimensions=3, n_layers=3):
        super(autoNN, self).__init__()
        self.n_inputs = n_inputs
        self.n_dimensions = n_dimensions
        self.n_layers = n_layers
        self.topology = get_topology(n_inputs, n_layers, n_dimensions)
        self.encoder = nn.ModuleList()
        self.decoder = nn.ModuleList()
        print(self.topology)
        for i in range(n_layers):
            if i == 0:
                self.encoder.append(nn.Linear(n_inputs, self.topology[i+1]))
            else:
                self.encoder.append(nn.Linear(self.topology[i], self.topology[i+1]))
            self.encoder.append(nn.ReLU())
        self.encoder = self.encoder[:-1]
        # invert the encoder using the encoder object
        for i in self.encoder[::-1]:
            if isinstance(i, nn.Linear):
                self.decoder.append(nn.Linear(i.out_features, i.in_features))
                self.decoder.append(nn.ReLU())
        self.decoder = self.decoder[:-1]
                   
    
    def forward(self, x):
        # encode the input
        for layer in self.encoder:
            x = layer(x)
        # decode the input
        for layer in self.decoder:
            x = layer(x)
        return x
    
    def encode(self, x):
        x = x.reshape(-1, self.n_inputs)
        print(x.shape)
        for layer in self.encoder:
            x = layer(x)
        return x
    def decode(self, x):
        x = x.reshape(-1, self.n_inputs)
        for layer in self.decoder:
            x = layer(x)
        return x

In [12]:
# append y data to the end of the x data
X_to_code = torch.cat((X_train, y_train), dim=1)
auto = autoNN(n_inputs=n_features_train+1, n_dimensions=3, n_layers=3)
auto


[58, 29, 14, 3]


autoNN(
  (encoder): ModuleList(
    (0): Linear(in_features=58, out_features=29, bias=True)
    (1): ReLU()
    (2): Linear(in_features=29, out_features=14, bias=True)
    (3): ReLU()
    (4): Linear(in_features=14, out_features=3, bias=True)
  )
  (decoder): ModuleList(
    (0): Linear(in_features=3, out_features=14, bias=True)
    (1): ReLU()
    (2): Linear(in_features=14, out_features=29, bias=True)
    (3): ReLU()
    (4): Linear(in_features=29, out_features=58, bias=True)
  )
)

In [13]:
losses = kmk.run_Pytorch(auto, X_to_code, X_to_code, n_epochs=1000, batch_size=1000, learning_rate=1e-3)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx